In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Papillary_Cell_Carcinoma"
cohort = "GSE40914"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma/GSE40914"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE40914.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE40914.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE40914.csv"
json_path = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression analysis and in silico characterization of intronic long noncoding RNAs in renal cell carcinoma: emerging functional associations"
!Series_summary	"Intronic and intergenic long noncoding RNAs (lncRNAs) are emerging gene expression regulators. The molecular pathogenesis of renal cell carcinoma (RCC) is still poorly understood, and in particular, limited studies are available for intronic lncRNAs expressed in RCC. Microarray experiments were performed with two different custom-designed arrays enriched with probes for lncRNAs mapping to intronic genomic regions. Samples from 18 primary clear cell RCC tumors and 11 nontumor adjacent matched tissues were analyzed with 4k-probes microarrays. Oligoarrays with 44k-probes were used to interrogate 17 RCC samples (14 clear cell, 2 papillary, 1 chromophobe subtypes) split into four pools. Meta-analyses were performed by taking the genomic coordinates of the RCC-expressed lncRNAs, and cross-referenc

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains expression data for lncRNAs and possibly mRNAs
# The Series_summary mentions "microarray experiments" and expression analysis, indicating gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary, we can see:
# - Key 0 contains disease information (renal cell carcinoma)
# - Key 1 contains tissue information (kidney tumor)
# - Key 2 contains sample type (pool)

# For trait (Kidney_Papillary_Cell_Carcinoma):
# The disease field indicates renal cell carcinoma, but we need to confirm if it's papillary type
# From the background information, we see "17 RCC samples (14 clear cell, 2 papillary, 1 chromophobe subtypes)"
# This confirms the dataset includes papillary RCC samples, but all samples in this cohort are labeled generically as RCC
trait_row = 0  # Using the disease field to extract trait

# For age:
# There is no age information in the sample characteristics
age_row = None

# For gender:
# There is no gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert RCC value to binary indicator for Kidney Papillary Cell Carcinoma.
    From the background info, we know there are 2 papillary RCC samples out of 17 total RCC samples.
    Since the data is pooled and we can't distinguish which samples are papillary type,
    we'll consider all RCC samples as potentially containing papillary RCC.
    """
    if not value:
        return None
    
    # Extract the value part after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value indicates RCC
    if 'renal cell carcinoma' in value.lower() or 'rcc' in value.lower():
        return 1  # Treat all RCC samples as potentially containing papillary type
    elif 'normal' in value.lower() or 'nontumor' in value.lower() or 'non-tumor' in value.lower():
        return 0  # Non-tumor samples
    else:
        return None  # Unknown

# Age and gender conversion functions are not needed as the data is not available

# 3. Save Metadata
# We have trait data but it's generically labeled as RCC, not specifically papillary
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the function from the library
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=None,
        gender_row=gender_row,
        convert_gender=None
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save the clinical features to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview:
{'GSM1004655': [nan], 'GSM1004656': [nan], 'GSM1004657': [nan], 'GSM1004658': [nan], 'GSM1004659': [nan], 'GSM1004660': [nan], 'GSM1004661': [nan], 'GSM1004662': [nan], 'GSM1004663': [nan], 'GSM1004664': [nan], 'GSM1004665': [nan], 'GSM1004666': [nan], 'GSM1004667': [nan], 'GSM1004668': [nan], 'GSM1004669': [nan], 'GSM1004670': [nan], 'GSM1004671': [nan], 'GSM1004672': [nan], 'GSM1004673': [nan], 'GSM1004674': [nan], 'GSM1004675': [nan], 'GSM1004676': [nan], 'GSM1004677': [nan], 'GSM1004678': [nan], 'GSM1004679': [nan], 'GSM1004680': [nan], 'GSM1004681': [nan], 'GSM1004682': [nan], 'GSM1004683': [nan], 'GSM1004684': [nan], 'GSM1004685': [nan], 'GSM1004686': [nan], 'GSM1004687': [nan], 'GSM1004688': [nan], 'GSM1004689': [nan], 'GSM1004690': [nan], 'GSM1004691': [nan], 'GSM1004692': [nan], 'GSM1004693': [nan], 'GSM1004694': [nan], 'GSM1004695': [nan], 'GSM1004696': [nan], 'GSM1004697': [nan], 'GSM1004698': [nan], 'GSM1004699': [nan], 'GSM1004700': [nan], 'GSM10

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Expression analysis and in silico characterization of intronic long noncoding RNAs in renal cell carcinoma: emerging functional associations"
Line 1: !Series_geo_accession	"GSE40914"
Line 2: !Series_status	"Public on Jan 21 2014"
Line 3: !Series_submission_date	"Sep 15 2012"
Line 4: !Series_last_update_date	"Apr 21 2015"
Line 5: !Series_pubmed_id	"24238219"
Line 6: !Series_summary	"Intronic and intergenic long noncoding RNAs (lncRNAs) are emerging gene expression regulators. The molecular pathogenesis of renal cell carcinoma (RCC) is still poorly understood, and in particular, limited studies are available for intronic lncRNAs expressed in RCC. Microarray experiments were performed with two different custom-designed arrays enriched with probes for lncRNAs mapping to intronic genomic regions. Samples from 18 primary clear cell RCC tumors and 11 nontumor adjacent matched tissues were analyzed with 4k-probes microarrays. Oligoarray

### Step 4: Gene Identifier Review

In [5]:
# Examine the first 20 gene identifiers shown above
# These appear to be just numeric values (1, 2, 3, etc.) rather than standard gene symbols
# This indicates they are likely probe IDs that need to be mapped to gene symbols

# Based on biomedical knowledge, human gene symbols follow specific naming conventions
# They are typically a combination of letters and sometimes numbers (e.g., TP53, BRCA1, ESR1)
# The identifiers shown are purely numeric, suggesting they are probe IDs or some other identifier

# Since we need proper gene symbols for biological interpretation
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 463119 rows

Gene annotation preview (first few rows):
{'ID': ['910', '4260', '1981', '2381', '4288'], 'GB_ACC': ['BE833259', 'BE702227', 'BF364095', 'BE081005', 'AW880607'], 'SPOT_TYPE': ['Exonic', 'Exonic', 'Exonic', 'Exonic', 'Exonic'], 'GENE_ID': ['85439', '2776', '84131', '2776', '54768'], 'GENE_SYMBOL': ['STON2', 'GNAQ', 'CEP78', 'GNAQ', 'HYDIN'], 'GENE_ANNOTATION': ['stonin 2', 'Guanine nucleotide binding protein (G protein), q polypeptide', 'centrosomal protein 78kDa', 'Guanine nucleotide binding protein (G protein), q polypeptide', 'hydrocephalus inducing homolog 2 (mouse); hydrocephalus inducing homolog (mouse)'], 'CPC_CODING_POTENTIAL': ['noncoding', 'noncoding', 'noncoding', 'noncoding', '-'], 'SEQUENCE': ['CTGATCCGCTTAAGCTTAGTATGTTTGAGTGTGTAATTTTAGTTTCTTTTCTGGTTGTATTTGTGGTAGTCAGATGTGTTGGATTGATTCCAACTGGACAGAGTAAGGAATTCCAGCATCCTCTTCCTGCTTGCTCGTGTTACCCCACAGATCAAACCCTCAATTCTAGTTGGGGATGCTGTCTAGCCCCACACCATGACTGAAGCCTTAAGCACTGTTGCG

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene_annotation correspond to probe IDs and gene symbols
# From the gene annotation preview, we can see:
# - 'ID' column contains numeric identifiers (matching gene expression data's index)
# - 'GENE_SYMBOL' column contains gene symbols (e.g., STON2, GNAQ)

print("Creating gene mapping from probe IDs to gene symbols...")

# 2. Create the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Created gene mapping with {len(gene_mapping)} entries")
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Ensure gene expression data has numeric values before applying mapping
print("\nEnsuring gene expression data has numeric values...")
# Convert all columns (except ID if it exists) to numeric types
for col in gene_data.columns:
    gene_data[col] = pd.to_numeric(gene_data[col], errors='coerce')

print(f"Gene data shape before mapping: {gene_data.shape}")
print("Data types in gene_data after conversion:")
print(gene_data.dtypes.head())

# 4. Apply the gene mapping to convert probe-level data to gene expression data
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Created gene expression data with {len(gene_data.index)} genes")
print("Gene expression data preview (first few rows):")
print(preview_df(gene_data))

# Save the gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Creating gene mapping from probe IDs to gene symbols...
Created gene mapping with 45204 entries
Gene mapping preview:
{'ID': ['910', '4260', '1981', '2381', '4288'], 'Gene': ['STON2', 'GNAQ', 'CEP78', 'GNAQ', 'HYDIN']}

Ensuring gene expression data has numeric values...
Gene data shape before mapping: (3275, 76)
Data types in gene_data after conversion:
GSM1004655    float64
GSM1004656    float64
GSM1004657    float64
GSM1004658    float64
GSM1004659    float64
dtype: object

Converting probe-level measurements to gene expression data...
Created gene expression data with 2645 genes
Gene expression data preview (first few rows):
{'GSM1004655': [11.611, 10.926954546000001, 5.866136364, 0.0, 2.304568182], 'GSM1004656': [12.39861364, 15.123068182, 6.7795, 0.0, 2.531636364], 'GSM1004657': [8.044136364, 9.524568182, 7.152886364, 0.0, 4.169409091], 'GSM1004658': [8.556590909, 9.290693181, 7.174977273, 0.0, 3.884727273], 'GSM1004659': [7.704806818, 16.374318182, 5.923522727, 0.0, 0.992409091]

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Load gene data that was saved in step 6
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
    
    # Normalize gene symbols using NCBI Gene database
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {gene_data.shape[0]} genes")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    # Load clinical data from step 2
    clinical_data = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_data.shape}")
    
    # Convert clinical data to proper format for linking (need to transpose for geo_link_clinical_genetic_data)
    clinical_transpose = clinical_data.set_index(trait).T
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_transpose, gene_data)
    print(f"Created linked data with {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    is_trait_available = True
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    is_trait_available = False
    linked_data = pd.DataFrame()

# 3. Handle missing values in the linked data
if is_trait_available and not linked_data.empty:
    print("\nHandling missing values...")
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        
    # 4. Determine whether the trait and demographic features are biased
    print("\nEvaluating feature bias...")
    try:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Trait bias determination: {is_biased}")
        print(f"Final linked data shape: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error evaluating feature bias: {e}")
        is_biased = True
else:
    print("\nSkipping missing value handling and bias evaluation as linked data is not available")
    is_biased = True

# 5. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_trait_available:
    note = "Dataset does not contain required trait information"
elif is_biased:
    note = "Dataset has severe bias in the trait distribution"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Final linked data saved to {out_data_file}")
else:
    if note:
        print(f"Reason: {note}")
    else:
        print("Dataset does not meet quality criteria for the specified trait")


Normalizing gene symbols...
Loaded gene data with 2645 genes and 76 samples


After normalization: 2629 genes
Normalized gene data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE40914.csv

Linking clinical and genetic data...
Loaded clinical data with shape: (1, 76)
Error linking clinical and genetic data: "None of ['Kidney_Papillary_Cell_Carcinoma'] are in the columns"

Skipping missing value handling and bias evaluation as linked data is not available

Performing final validation...
Abnormality detected in the cohort: GSE40914. Preprocessing failed.

Dataset usability for Kidney_Papillary_Cell_Carcinoma association studies: False
Reason: Dataset does not contain required trait information
